In [1]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.Map.Strict as M
import qualified Data.ByteString as B
import qualified Data.ByteString.Char8 as B (lines)

input <- B.lines <$> B.readFile "../input/day8.txt"

coords = (,) <$> [0..49] <*> [0..5]

grid = M.fromList $ zip coords (repeat False)

In [5]:
import qualified Data.Attoparsec.ByteString       as A
import qualified Data.Attoparsec.ByteString.Char8 as AC

data Instruction = Rec Int Int | Row Int Int | Col Int Int deriving (Eq, Show)

parseRec = do
    A.string "rect"
    AC.space
    x <- AC.decimal
    AC.char 'x'
    y <- AC.decimal
    return $ Rec x y

parseRow = do
    A.string "rotate"
    AC.space
    A.string "row"
    AC.space
    A.string "y="
    y <- AC.decimal
    AC.space
    AC.string "by"
    AC.space
    r <- AC.decimal
    return $ Row y r

parseCol = do
    A.string "rotate"
    AC.space
    A.string "column"
    AC.space
    A.string "x="
    x <- AC.decimal
    AC.space
    AC.string "by"
    AC.space
    r <- AC.decimal
    return $ Col x r

parseInstr = A.choice [parseRec, parseRow, parseCol]

parsed = map (either error id . A.parseOnly parseInstr) input
head parsed

Rec 1 1

In [22]:
execute (Rec x y) _ = M.fromList $ zip ((,) <$> [0..x-1] <*> [0..y-1]) (repeat True)
execute (Row y r) m = let
    row  = M.toList $ M.filterWithKey (\(_,y') _ -> y'==y) m
    row' = map (\((x,y),v) -> (((x+r) `mod` 50,y),v)) row
    in M.fromList row'
execute (Col x r) m = let
    col  = M.toList $ M.filterWithKey (\(x',_) _ -> x'==x) m
    col' = map (\((x,y),v) -> ((x,(y+r) `mod` 6),v)) col
    in M.fromList col'

run m []     = m
run m (i:is) = let
    changes = execute i m
    m'      = M.union changes m
    in run m' is

result = run grid parsed

length $ filter id $ M.elems result

106

In [28]:
charAt c m = if m M.! c then '#' else '.'

mapM_ putStrLn [[ charAt (x,y) result |x <- [0..49]] | y <- [0..5]]

.##..####.#....####.#.....##..#...#####..##...###.
#..#.#....#....#....#....#..#.#...##....#..#.#....
#....###..#....###..#....#..#..#.#.###..#....#....
#....#....#....#....#....#..#...#..#....#.....##..
#..#.#....#....#....#....#..#...#..#....#..#....#.
.##..#....####.####.####..##....#..#.....##..###..